In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
!pip install arch
from arch import arch_model

     |████████████████████████████████| 808kB 4.0MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
spot=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/spot_price/^NSEI.csv')
spot['return']=spot['Adj Close'].pct_change(1)
spot['volatility']=spot['return'].rolling(20).std()
spot['volatility']=spot['volatility']*np.sqrt(252)
spot.drop(['Open','High','Low' ,'Close','Volume'],axis=1,inplace=True)


In [ ]:
del spot['Adj Close']
del spot['volatility']
spot.set_index('Date')


,return
Date,
2013-12-02,NaN
2013-12-03,-0.002573
2013-12-04,-0.006595
2013-12-05,0.013009
2013-12-06,0.003012
...,...
2020-12-23,0.010010
2020-12-24,0.010893
2020-12-28,0.009015


In [ ]:
x=[]
x=spot.iloc[1:]
del x['Date']
x

,return
1,-0.002573
2,-0.006595
3,0.013009
4,0.003012
5,0.016614
...,...
1741,0.010010
1742,0.010893
1743,0.009015
1744,0.004282


In [ ]:
ans=[]

model = arch_model(x.iloc[1:], mean='Constant', vol='GARCH', p=2, q=2)
# fit model
model_fit = model.fit()
yhat = model_fit.forecast(horizon=1745)


Iteration:      1,   Func. Count:      8,   Neg. LLF: -5747.374847209018
Iteration:      2,   Func. Count:     24,   Neg. LLF: -5748.118292698517
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -5748.118299596597
            Iterations: 2
            Function evaluations: 35
            Gradient evaluations: 2


/usr/local/lib/python3.7/dist-packages/arch/univariate/base.py:317: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001175. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning
/usr/local/lib/python3.7/dist-packages/arch/__future__/_utility.py:21: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  FutureWarning,


In [ ]:
yhat.variance.values[-1, :]

array([0.00010232, 0.00010126, 0.00010211, ..., 0.00011749, 0.00011749,
       0.00011749])

In [ ]:
print(ans)
x.iloc[20:30]

[array([0.13129383]), array([0.1298931]), array([0.12696977]), array([0.12296559]), array([0.12699555]), array([0.1344459]), array([0.13596803]), array([0.13790936]), array([0.1395666]), array([0.14051668]), array([0.12205179]), array([0.12240347]), array([0.11388044]), array([0.12351116]), array([0.12823164]), array([0.12499988]), array([0.13263089]), array([0.13283]), array([0.13631688]), array([0.13630479]), array([0.11760739]), array([0.11667405]), array([0.11720267]), array([0.11802117]), array([0.12112449]), array([0.11969347]), array([0.1166622]), array([0.11502242]), array([0.12735297]), array([0.13057737]), array([0.12633258]), array([0.12693164]), array([0.12962337]), array([0.11570904]), array([0.11485731]), array([0.11496084]), array([0.11594071]), array([0.11616891]), array([0.11104713]), array([0.11365149]), array([0.11597112]), array([0.11322206]), array([0.11717709]), array([0.11518351]), array([0.10363415]), array([0.09859726]), array([0.09770472]), array([0.09415083])

40    0.134733
41    0.134644
42    0.129603
43    0.130273
44    0.138636
45    0.138186
46    0.138797
47    0.139359
48    0.140642
49    0.124983
Name: volatility, dtype: float64

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
x1=np.array(np.sqrt(yhat.variance.values[-1, :])).reshape(-1,1)


In [ ]:
reg=LinearRegression().fit(x1,y1)
mae= mean_absolute_error(x1,y1)

NameError: ignored

In [ ]:
reg.score(x1,y1)

0.9810229929925433

In [ ]:
mae

0.006378285834036613

In [ ]:
call=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/call_black_scholes.csv')
put=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/put_black_scholes.csv')


In [ ]:
call.shape

(56654, 9)

In [ ]:
call.head(20)

,Date,Strike Price,Close,Expiration,Adj Close,return,volatility,Price,bs_call
0,2013-12-31,6200,199.65,30,6304.000000,0.002051,0.135503,8.820,188.621115
1,2013-12-31,6300,131.00,30,6304.000000,0.002051,0.135503,8.820,124.059028
2,2013-12-31,6400,79.00,30,6304.000000,0.002051,0.135503,8.820,75.195204
3,2013-12-31,6500,41.35,30,6304.000000,0.002051,0.135503,8.820,41.683668
4,2013-12-31,6600,18.60,30,6304.000000,0.002051,0.135503,8.820,21.015752
5,2014-01-02,6200,136.40,28,6221.149902,-0.013142,0.141609,8.840,131.617183
6,2014-01-02,6300,85.00,28,6221.149902,-0.013142,0.141609,8.840,80.554650
7,2014-01-02,6400,48.95,28,6221.149902,-0.013142,0.141609,8.840,45.102228
8,2014-01-02,6500,25.45,28,6221.149902,-0.013142,0.141609,8.840,22.966379
9,2014-01-02,6600,11.70,28,6221.149902,-0.013142,0.141609,8.840,10.595069


In [ ]:
call[call['Date']=='2014-1-']

(7006, 4)

In [ ]:
call[call['Strike price']==7750]

,Date,Strike price,Close,Expiration


In [ ]:
final=spot[spot['Date']>'2013-12-02']

In [ ]:
final

,Date,return
1,2013-12-03,-0.002573
2,2013-12-04,-0.006595
3,2013-12-05,0.013009
4,2013-12-06,0.003012
5,2013-12-09,0.016614
...,...,...
1741,2020-12-23,0.010010
1742,2020-12-24,0.010893
1743,2020-12-28,0.009015
1744,2020-12-29,0.004282


In [ ]:
x1[0][0]

0.010218490561695557

In [ ]:
for i in range(0,1745):
  final.iloc[i,1]=x1[i][0]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
final.head()

,Date,return
1,2013-12-03,0.010115
2,2013-12-04,0.010063
3,2013-12-05,0.010105
4,2013-12-06,0.010100
5,2013-12-09,0.010117


In [ ]:
final['Garch_Volatility']=final['return']
del final['return']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
result = pd.merge(call, final, 
                   on='Date', 
                   how='inner')

In [ ]:
result.head(30)

,Date,Strike Price,Close,Expiration,Adj Close,return,volatility,Price,bs_call,Garch_Volatility
0,2013-12-31,6200,199.65,30,6304.000000,0.002051,0.135503,8.820,188.621115,0.010172
1,2013-12-31,6300,131.00,30,6304.000000,0.002051,0.135503,8.820,124.059028,0.010172
2,2013-12-31,6400,79.00,30,6304.000000,0.002051,0.135503,8.820,75.195204,0.010172
3,2013-12-31,6500,41.35,30,6304.000000,0.002051,0.135503,8.820,41.683668,0.010172
4,2013-12-31,6600,18.60,30,6304.000000,0.002051,0.135503,8.820,21.015752,0.010172
5,2014-01-02,6200,136.40,28,6221.149902,-0.013142,0.141609,8.840,131.617183,0.010172
6,2014-01-02,6300,85.00,28,6221.149902,-0.013142,0.141609,8.840,80.554650,0.010172
7,2014-01-02,6400,48.95,28,6221.149902,-0.013142,0.141609,8.840,45.102228,0.010172
8,2014-01-02,6500,25.45,28,6221.149902,-0.013142,0.141609,8.840,22.966379,0.010172
9,2014-01-02,6600,11.70,28,6221.149902,-0.013142,0.141609,8.840,10.595069,0.010172


In [ ]:
final.head()

,Date,Garch_Volatility
40,2014-01-28,0.132432
41,2014-01-29,0.131952
42,2014-01-30,0.131423
43,2014-01-31,0.127339
44,2014-02-03,0.127654


In [ ]:
result.to_csv('garch_call_today.csv')

In [ ]:
import os
os.chdir('/content/')